## Análisis de sentimientos de Twitter con Python
* https://platzi.com/tutoriales/1874-python-lenguaje-natural/5654-realiza-un-analisis-de-sentimiento-en-3-pasos-con-python/
    * Ejemplo https://twitter.com/whaleandjaguar_?lang=en
* https://towardsdatascience.com/my-absolute-go-to-for-sentiment-analysis-textblob-3ac3a11d524
* https://www.justintodata.com/twitter-sentiment-analysis-python/
* https://www.pluralsight.com/guides/building-a-twitter-sentiment-analysis-in-python

### 1) Descargar tweets

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
pd.set_option('display.max_colwidth',500)

In [3]:
df=pd.read_json('https://raw.githubusercontent.com/restrepo/twitter/main/tweets_df.json')

### Cree un nuevo clasificador
Vamos 
https://textblob.readthedocs.io/en/dev/classifiers.html#classifiers

* A favor del Paro → `'fav'`
* En contra del Paro→ `'con'`
* Informativo → `'new'`

Para crear los conjutos de datos para training y para test debemos clasisificar un conjunto suficientemente grande de textos de tweets en las dos categorias en números similares para cada una de las dos categorias

Crearemos un algoritmo que nos permite clasificar algunos tweets del DataFrame en una nueva columna `'label'`, la cual tendrá un valor nulo cuando el tweet no este clasificado. Una vez un tweet sea clasificado, se añadirá a una lista de similaridad y sólo se clasificarán nuevos tweets que no sean similares a los previamente clasificados.  Para ello se usará el método `extractOne` del módulo `process` de `fuzzywuzzy` con scorer `fuzz.ratio` basado en la distancia Levenshtein entre dos textos.

In [5]:
from fuzzywuzzy import process
from fuzzywuzzy import  fuzz

In [38]:
n_train=10
n_test=5
n_traintest=n_train+n_test
df['label']=None
ii=0
similarity=[]
for i in df.index:
    print("="*80)
    tweet=df.loc[i,'text']
    if similarity:
        if process.extractOne(tweet,similarity,scorer=fuzz.ratio)[-1]<90:
            similarity.append(tweet)
            posneg=input(f"{tweet}: https://twitter.com/twitter/status/{df.loc[i,'id']}\n → f/c/n/s para 'a favor'/'en contra'/'neutro'/'spam' or <Enter> para continuar:\n")
            if posneg=='c':
                posneg='con'
            elif posneg=='f':
                posneg='fav'
            elif posneg=='n':
                posneg='neu'
            elif posneg=='s':
                posneg='spa'
            else:
                continue
            df.loc[i,'label']=posneg
            ii=ii+1
    else:
        similarity.append(tweet)

    if ii==n_traintest:
        break

RT @PoloDemocratico: El #ParoNacional es la manifestación del pueblo por la inconformidad qué hay con el gobierno de @IvanDuque, queremos q…: https://twitter.com/twitter/status/1394753529844219907
 → f/c/n/s para 'a favor'/'en contra'/'neutro'/'spam' or <Enter> para continuar:
 f


RT @PrensaRural: #ParoNacional, Campesinado del Catatumbo:

En alerta máxima ante orden de Duque de tierra arrasada

👉https://t.co/31jO1zLD…: https://twitter.com/twitter/status/1394753517546508288
 → f/c/n/s para 'a favor'/'en contra'/'neutro'/'spam' or <Enter> para continuar:
 


RT @Tiradorsilencio: Como Ciudadano Colombiano me solidarizo y Apoyo d manera Absoluta y Total; la Carta d Ciudadanos d #Cali pidiendo d fo…: https://twitter.com/twitter/status/1394753499943079939
 → f/c/n/s para 'a favor'/'en contra'/'neutro'/'spam' or <Enter> para continuar:
 


RT @alertaroja234: @BluRadioCo Para aquellos que llaman a la guerra y al uso desproporcionado de la fuerza. Aquellos que bloquean son en la…: https://twitter.com/twitter/status/1394753497912942594
 → f/c/n/s para 'a favor'/'en contra'/'neutro'/'spam' or <Enter> para continuar:
 


RT @madeleintlSUR: Un día de #ParoNacional #Medellin #Colombia de lee desde el aire anti #uribista https://t.co/cmX982EmFR: https://twitter.com/twitter/status/1394753451314253828
 → f/c/n/s para 'a favor'/'en contra'/'neutro'/'spam' or <Enter> para continuar:
 


...

RT @FdoQuijano: #ParoNacional 🇨🇴😁 #NosRecuerda @HELIODOPTERO "El paro nacional despierta emoción y simpatía en Miss Universo." https://t.co…: https://twitter.com/twitter/status/1394751617845637120
 → f/c/n/s para 'a favor'/'en contra'/'neutro'/'spam' or <Enter> para continuar:
 


RT @NSimpsombia: Como realmente abatieron a Jesús Santrich
- Sr Thompson 

#JesusSantrich #Santrich #Farc #Colombia #paronacional https://t…: https://twitter.com/twitter/status/1394751593522868230
 → f/c/n/s para 'a favor'/'en contra'/'neutro'/'spam' or <Enter> para continuar:
 


RT @alejzuluaga: .@juanpisrules, @AlejandroRia, hicimos una breve adaptación de su maravilloso video. Escena protagonizada por el alcalde d…: https://twitter.com/twitter/status/1394751585582977025
 → f/c/n/s para 'a favor'/'en contra'/'neutro'/'spam' or <Enter> para continuar:
 


RT @PBIcanada: Este es un vehículo de fabricación canadiense que fue vendido a la policía nacional de Colombia. Estamos monitoreando las pu…: https://twitter.com/twitter/status/1394751572157116423
 → f/c/n/s para 'a favor'/'en contra'/'neutro'/'spam' or <Enter> para continuar:
 


Inicia tu carrera con nosotros, sé parte de nuestras capacitaciones y déjate asesorar y guiar por nuestros expertos. 

Abrimos nuevos #cupos Escríbenos 322 8864081

#cali #colombia #trabajoencali #trabajocali #modelaje #aumentatusingresos #martes #empleo #empleoencali #elempleo https://t.co/eXKrz5Lv2z: https://twitter.com/twitter/status/1394751568608649218
 → f/c/n/s para 'a favor'/'en contra'/'neutro'/'spam' or <Enter> para continuar:
 s


Select the train/test filtered DataFrame

In [39]:
tmp=df[~df['label'].isna()].reset_index(drop=True)

In [40]:
tmp[:3]

,user_name,user_location,user_description,user_verified,date,text,hashtags,source,id,label,test,prob
0,¡#55/86 en 2022!🦍🦍💪💎💎👏👏🚀🚀🌕,,#sigamonoslosbuenos #movimientoantiuribista,False,2021-05-18 20:35:29,"RT @PoloDemocratico: El #ParoNacional es la manifestación del pueblo por la inconformidad qué hay con el gobierno de @IvanDuque, queremos q…",[ParoNacional],Twitter for iPhone,1394753529844219907,fav,fav,0.73
1,YASSER AMIN® (turko amin)®,por ahi,cuenta nueva #12-\npublicista\nfotógrafo amateur\nsoltero,False,2021-05-18 20:34:42,@Emile1985 @elespectador No pues los \nmuch4chosdrogos y d3lincu3nt3s es lo que hay en ese #ParoNacional,[ParoNacional],Twitter for Android,1394753334536458242,con,fav,0.72
2,Carlos Guerrero,,,False,2021-05-18 20:34:39,"RT @aciprensa: El delegado especial del @episcopadocol, Mons. Héctor Henao, pidió que se excluya todo gesto de violencia en Colombia, de ta…",None,Twitter for Android,1394753319281778688,neu,con,0.91


In [41]:
tmp.shape

(15, 12)

In [42]:
train=tmp[:n_train][['text','label']].reset_index(drop=True)

In [43]:
from textblob.classifiers import NaiveBayesClassifier

In [44]:
import nltk
nltk.download('punkt')  

[nltk_data] Downloading package punkt to /home/restrepo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [45]:
cl = NaiveBayesClassifier(  [ (d.get('text'),d.get('label')) for d in test.to_dict(orient='records')]  )

In [46]:
fulltest=test.copy()
fulltest['test']=fulltest['text'].apply(cl.classify)

In [47]:
fulltest[['text','label','test']]

,text,label,test
0,[IRI·Prensa·13/05/21] Diálogo sincero y constructivo ante crisis nacional. https://t.co/KvAqsgwfsS #ParoNacional #Colombia @CIDH @infopresidencia @MinInterior @faiths4forests @brasiliri @gaiaamazonas @InstitutoSINCHI @cutcolombia,fav,fav
1,"RT @boyacasietedias: El #Sondeo7días de hoy: Cuando ya se cumplen 21 días de #ParoNacional, a propósito de la medida anunciada por el Presi…",neu,neu
2,"@BluRadioCo Por supuesto que sí, más cuando quienes bloquean, saquean y violan derechos de las grandes mayorías de Colombia #NoMasParo #NoMasBloqueos #ElParoNoMeRepresenta #AtrabajarMamertos",con,con
3,"RT @teleSURtv: Los primeros grupos de la #MingaIndígena llegaron este martes a la ciudad de #Medellín, #Colombia🇨🇴, para apoyar el #ParoNac…",fav,fav
4,"RT @Caprietto: Algunos se molestan, pero hay que decirll:\n\n— El pueblo no salió a la calle\n— Bloqueas una calle, sos un delincuente\n— Los d…",con,con


In [48]:
cl.accuracy( [ (d.get('text'),d.get('label')) for d in test.to_dict(orient='records')] )

1.0

In [49]:
fulltest['prob']=fulltest['text'].apply(lambda t:   
                                cl.prob_classify( t ).prob(   cl.classify( t  )  ) ).round(2)

In [50]:
fulltest

,text,label,test,prob
0,[IRI·Prensa·13/05/21] Diálogo sincero y constructivo ante crisis nacional. https://t.co/KvAqsgwfsS #ParoNacional #Colombia @CIDH @infopresidencia @MinInterior @faiths4forests @brasiliri @gaiaamazonas @InstitutoSINCHI @cutcolombia,fav,fav,1.0
1,"RT @boyacasietedias: El #Sondeo7días de hoy: Cuando ya se cumplen 21 días de #ParoNacional, a propósito de la medida anunciada por el Presi…",neu,neu,1.0
2,"@BluRadioCo Por supuesto que sí, más cuando quienes bloquean, saquean y violan derechos de las grandes mayorías de Colombia #NoMasParo #NoMasBloqueos #ElParoNoMeRepresenta #AtrabajarMamertos",con,con,1.0
3,"RT @teleSURtv: Los primeros grupos de la #MingaIndígena llegaron este martes a la ciudad de #Medellín, #Colombia🇨🇴, para apoyar el #ParoNac…",fav,fav,1.0
4,"RT @Caprietto: Algunos se molestan, pero hay que decirll:\n\n— El pueblo no salió a la calle\n— Bloqueas una calle, sos un delincuente\n— Los d…",con,con,1.0


Aplicar al DataFrame Completo

In [51]:
df['test']=df['text'].apply(cl.classify)
df['prob']=df['text'].apply(lambda t:   
                                cl.prob_classify( t ).prob(   cl.classify( t  )  ) ).round(2)

In [52]:
dfp=df.drop_duplicates('text').reset_index(drop=True)

In [53]:
dfp[['text','test','prob']].sample(10)

,text,test,prob
1245,"https://t.co/m0sRyIqyvR \nAyúdeme reportando el perfil de esta #Pendeja, que tiene aires de superioridad, creyéndose mexicana e insultando a los manifestantes. #18DeMayo #ParoNacional #ColombiaResiste",fav,1.00
1695,"RT @Cali24Horas: #Movilidad | @MiTerminalCali informó que únicamente hay despachos activos con destino a #Jamundí, #Santander de Quilichao,…",con,0.66
89,"El uribista promedio cree que la gente de a pie es igual de tonta y arrodillada que él, que sale a marchar porque sus dirigentes le dicen. Por eso es que salen a buscar el culpable, el lider de las marchas y del #ParoNacional.",con,0.66
1168,"Quiere las vías libres, para ingresar armamento o pa putas #DuqueRenuncieYa #NoticiasCaracol #ParoNacional https://t.co/7EM7gO4Izz",fav,1.00
1345,@rcnradio Primer vez en #Colombia que un #ParoNacional dura 21 días adornado con infames bloqueos y vandalismo! Duque permite la dictadura de bloqueos y vandalismo! #Duquedespierte @CNNEE,fav,0.72
1312,@Nessnio @IvanDuque Esta es polombia. #ParoNacional \n#SOSColombiaEnDictadura,fav,1.00
1666,#URGENTE A esta hora se presentan detenciones arbitrarias por parte de la fuerza pública hacia manifestantes y la Primera línea de Portal Américas. \n\n#bogota #Colombia #paronacional https://t.co/GiTSgSE30S,fav,0.98
1313,RT @juanfelipedelg8: @VickyDavilaH Nadie la interesa la notícia de su muerte porque el fue un traidor del proceso de paz y decidió seguir e…,fav,0.98
552,"Y pretenden cocinar “acuerdos"" #YumboResiste\n#ParoNacional18M #ParoAsesino #ParoNacionalColombia #ParoNacionalIndefinido #BogotaEnAlertaRoja #PopayanEsColombia #Cali #Medellin #Pereira #Bogota #Barranquilla #Valledupar #ValleDelCauca #Bucaramanga #Buga #siloecali https://t.co/zN7GljaMHn",fav,1.00
67,"RT @wdelbar: Las #MadresContraLaImpunidad se mantienen en las calles. ¡Vamos con todas, con todos, vamos con ellas! #ParoNacional #ParoNaci…",fav,0.93


In [54]:
dfp.groupby('test')['test'].count()

test
con     256
fav    1495
neu       2
Name: test, dtype: int64

Sin quitar RT

In [55]:
df.groupby('test')['test'].count()

test
con     592
fav    4404
neu       4
Name: test, dtype: int64